In [77]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

In [78]:
import warnings
warnings.filterwarnings("ignore")

In [80]:
PRED = False

In [81]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/LSTM/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

C-LSTM_1D_19F_Class_03_Test.csv
C-LSTM_1D_19F_Class_03_Train.csv
C-LSTM_1H_0F_Class_Test_Results_01.csv
C-LSTM_1H_0F_Class_Test_Results_02.csv
C-LSTM_1H_0F_Class_Train_Results_01.csv
C-LSTM_1H_0F_Class_Train_Results_02.csv


In [82]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[1], header=0)
print(DATASET.head(20))
print(DATASET.shape)


       Open     High      Low    Close        Volume  Real_Class  \
0   1.20780  1.21167  1.20465  1.21079  1.126558e+06        -2.0   
1   1.21063  1.21349  1.20005  1.20102  1.065118e+06         0.0   
2   1.20131  1.20189  1.20019  1.20096  7.733910e+04         1.0   
3   1.20087  1.20685  1.19860  1.20468  1.124832e+06         2.0   
4   1.20455  1.21946  1.20442  1.21778  1.117748e+06         1.0   
5   1.21754  1.22208  1.21466  1.21889  1.106444e+06        -1.0   
6   1.21899  1.21999  1.21408  1.21666  1.132577e+06        -2.0   
7   1.21663  1.21931  1.20414  1.20558  1.063896e+06         1.0   
8   1.20628  1.20701  1.20509  1.20662  7.789710e+04         0.0   
9   1.20657  1.20802  1.20459  1.20650  1.110041e+06         1.0   
10  1.20640  1.21142  1.20518  1.20860  1.116064e+06         2.0   
11  1.20851  1.21929  1.20256  1.21814  1.108055e+06         2.0   
12  1.21808  1.22491  1.21587  1.22336  1.122153e+06         1.0   
13  1.22333  1.23084  1.21984  1.22690  1.056604

In [83]:
if (PRED):
    DATASET['Position'] = DATASET['Predicted_Class']
    DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
    DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
    DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
    DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
else:
    DATASET['Position'] = DATASET['Real_Class']
    DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
    DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
    DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
    DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
DATASET.head(10)

,Open,High,Low,Close,Volume,Real_Class,Predicted_Class,Position
0,1.20780,1.21167,1.20465,1.21079,1.126558e+06,-2.0,-2.0,-1.0
1,1.21063,1.21349,1.20005,1.20102,1.065118e+06,0.0,0.0,0.0
2,1.20131,1.20189,1.20019,1.20096,7.733910e+04,1.0,-1.0,0.0
3,1.20087,1.20685,1.19860,1.20468,1.124832e+06,2.0,2.0,1.0
4,1.20455,1.21946,1.20442,1.21778,1.117748e+06,1.0,2.0,1.0
5,1.21754,1.22208,1.21466,1.21889,1.106444e+06,-1.0,2.0,-1.0
6,1.21899,1.21999,1.21408,1.21666,1.132577e+06,-2.0,-2.0,-1.0
7,1.21663,1.21931,1.20414,1.20558,1.063896e+06,1.0,0.0,1.0
8,1.20628,1.20701,1.20509,1.20662,7.789710e+04,0.0,-2.0,0.0
9,1.20657,1.20802,1.20459,1.20650,1.110041e+06,1.0,-1.0,0.0


In [84]:
class limits(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        self.Op_Abierta = False
        
    def next(self):
        sig = self.data.Position[-1]
        if not self.Op_Abierta and sig >= self.buy_lim:
            self.buy()
            self.Op_Abierta = True
        if self.Op_Abierta and sig <= self.sell_lim:
            self.sell()
            self.Op_Abierta = False

In [48]:
class limits2(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        
    def next(self):
        sig = self.data.Position[-1]
        if  sig >= self.buy_lim:
            self.buy()
        if  sig <= self.sell_lim:
            self.sell()

In [33]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [85]:
DATASET['Predicted_Class'].value_counts()

 2.0    1233
-2.0    1042
 0.0     535
-1.0     490
 1.0     291
Name: Predicted_Class, dtype: int64

In [86]:
DATASET['Real_Class'].value_counts()

 2.0    798
-2.0    790
-1.0    700
 1.0    694
 0.0    609
Name: Real_Class, dtype: int64

In [75]:
bt = Backtest(DATASET, limits, cash=1000, commission=.0002)
bt.run()

Start                             0
End                           20303
Duration                      20303
Exposure [%]                99.9606
Equity Final [$]            94243.2
Equity Peak [$]             94243.2
Return [%]                  9324.32
Buy & Hold Return [%]       4.97299
Max. Drawdown [%]          -2.56114
Avg. Drawdown [%]        -0.0927908
Max. Drawdown Duration          125
Avg. Drawdown Duration      6.05505
# Trades                       3566
Win Rate [%]                 84.184
Best Trade [%]              2.35793
Worst Trade [%]            -2.09218
Avg. Trade [%]             0.127608
Max. Trade Duration              44
Avg. Trade Duration         5.69125
Expectancy [%]             0.160078
SQN                         19.0202
Sharpe Ratio                0.58714
Sortino Ratio              0.780489
Calmar Ratio              0.0498245
_strategy                    limits
dtype: object

bt.plot()